In [1]:
!pip install fuzzywuzzy distance

In [2]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np

# Tools for preprocessing input data
from bs4 import BeautifulSoup
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import re

import seaborn as sns
# Tools for creating ngrams and vectorizing input data
from gensim.models import Word2Vec, Phrases
from gensim.models import KeyedVectors
import matplotlib.pyplot as plt
from nltk.stem import PorterStemmer
#from bs4 import BeautifulSoup
from fuzzywuzzy import fuzz
import distance

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.optimizers import Adam
from keras.models import Model
from keras import backend as K
from keras.engine.topology import Layer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from tensorflow.keras import Model
from tensorflow.keras import Input
from tensorflow.keras.layers import Dense, Embedding,Bidirectional,LSTM,Attention,Concatenate,GlobalAveragePooling1D,Dropout
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from keras.layers.normalization import BatchNormalization

In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [4]:
train_df=pd.read_csv('/content/gdrive/MyDrive/Kaggle/train.csv')

In [5]:
train_df.shape

(404290, 6)

In [6]:
# Filling the null values with ' '
train_df = train_df.fillna('')

In [7]:
# Calculate question length and n_words
train_df['q1len'] = train_df['question1'].str.len() 
train_df['q2len'] = train_df['question2'].str.len()
train_df['q1_n_words'] = train_df['question1'].apply(lambda row: len(row.split(" ")))
train_df['q2_n_words'] = train_df['question2'].apply(lambda row: len(row.split(" ")))

In [8]:
def preprocessing(text, remove_stopwords=False, stem_words=False):
    # Clean the text, with the option to remove stopwords and to stem words.
    
    # Convert words to lower case and split them
    text = text.lower().split()

    # Optionally, remove stop words
    if remove_stopwords:
        stop_words = set(stopwords.words("english"))
        text = [w for w in text if not w in stop_words]
    
    text = " ".join(text)
    
    # Remove punctuation from text
    # text = "".join([c for c in text if c not in punctuation])

    # Clean the text
    text = re.sub(r"[^A-Za-z0-9^,!.\/'+-=]", " ", text)
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"can't", "cannot ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r",", " ", text)
    text = re.sub(r"\.", " ", text)
    text = re.sub(r"!", " ! ", text)
    text = re.sub(r"\/", " ", text)
    text = re.sub(r"\^", " ^ ", text)
    text = re.sub(r"\+", " + ", text)
    text = re.sub(r"\-", " - ", text)
    text = re.sub(r"\=", " = ", text)
    text = re.sub(r"'", " ", text)
    text = re.sub(r":", " : ", text)
    text = re.sub(r"(\d+)(k)", r"\g<1>000", text)
    text = re.sub(r" e g ", " eg ", text)
    text = re.sub(r" b g ", " bg ", text)
    text = re.sub(r" u s ", " american ", text)
    # text = re.sub(r"\0s", "0", text) # It doesn't make sense to me
    text = re.sub(r" 9 11 ", "911", text)
    text = re.sub(r"e - mail", "email", text)
    text = re.sub(r"j k", "jk", text)
    text = re.sub(r"\s{2,}", " ", text)
    
    # Optionally, shorten words to their stems
    if stem_words:
        text = text.split()
        stemmer = SnowballStemmer('english')
        stemmed_words = [stemmer.stem(word) for word in text]
        text = " ".join(stemmed_words)
    
    # Return a list of words
    return(text)

In [9]:
train_df['question1_clean'] = train_df['question1'].apply(preprocessing)
train_df['question2_clean'] = train_df['question2'].apply(preprocessing)

# Extract_Feautres

In [10]:
# get the Longest Common sub string

def get_longest_substr_ratio(a, b):
    strs = list(distance.lcsubstrings(a, b))
    if len(strs) == 0:
        return 0
    else:
        return len(strs[0]) / (min(len(a), len(b)) + 1)

# Extract fuzzy-wuzzy features
def extract_features (df):
    # The token sort approach involves tokenizing the string in question, sorting the tokens alphabetically, and 
    # then joining them back into a string We then compare the transformed strings with a simple ratio().
    df["token_sort_ratio"]      = df.apply(lambda x: fuzz.token_sort_ratio(x["question1"], x["question2"]), axis=1)
    df["fuzz_ratio"]            = df.apply(lambda x: fuzz.QRatio(x["question1"], x["question2"]), axis=1)
    df["fuzz_partial_ratio"]    = df.apply(lambda x: fuzz.partial_ratio(x["question1"], x["question2"]), axis=1)
    df["longest_substr_ratio"]  = df.apply(lambda x: get_longest_substr_ratio(x["question1"], x["question2"]), axis=1)
    return df

def normalized_word_share(row):
    w1 = set(map(lambda word: word.lower().strip().replace('?', ''), row['question1'].split(" ")))
    w2 = set(map(lambda word: word.lower().strip().replace('?', ''), row['question2'].split(" ")))    
    return 1.0 * len(w1 & w2)/(len(w1) + len(w2))

In [ ]:
train_df = extract_features(train_df)
train_df['word_share'] = train_df.apply(normalized_word_share, axis=1)

In [ ]:
for word in ['who', 'how', 'where', 'why', 'what', 'when', 'which']:
    train_df['have'+word+"_1"] = False
    train_df['have'+word+"_2"] = False
    train_df.loc[train_df.question1_clean.str.contains(r"\b"+word+"\\b", regex=True), ['have'+word+"_1"]] = True
    train_df.loc[train_df.question2_clean.str.contains(r"\b"+word+"\\b", regex=True), ['have'+word+"_2"]] = True
    train_df.loc[train_df.question1_clean.str.contains(r"\b"+word+"\\b", regex=True), ["question_word"+"_1"]] = word
    train_df.loc[train_df.question2_clean.str.contains(r"\b"+word+"\\b", regex=True), ["question_word"+"_2"]] = word

In [ ]:
train_df['have_same_q_w'] = False
train_df.loc[(train_df.question_word_1==train_df.question_word_2),['have_same_q_w']] = True

# Modelling

In [ ]:
#train_df.to_csv('train_preprocessed.csv', index = False)

In [ ]:
max_num_words = 200000
sequence_length = 60
embedding_dim = 300
validation_split_ratio = 0.1

In [ ]:
q1 = train_df['question1_clean'].values
q2 = train_df['question2_clean'].values
tokenizer = Tokenizer(num_words = max_num_words, split=' ', oov_token='<unw>')
tokenizer.fit_on_texts(q1+q2)

# this takes our sentences and replaces each word with an integer
X_1 = tokenizer.texts_to_sequences(q1)
X_2 = tokenizer.texts_to_sequences(q2)

In [ ]:
# we then pad the sequences so they're all the same length (sequence_length)
X_1 = pad_sequences(X_1, int(sequence_length))
X_2 = pad_sequences(X_2, int(sequence_length))

y = train_df['is_duplicate']

In [ ]:
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

In [ ]:
import pickle
embeddings_index = pickle.load( open( "/content/gdrive/MyDrive/embedding_matrix.p", "rb" ) )

In [ ]:
  embedding_matrix = np.zeros((max_num_words, embedding_dim))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector
print('Null word embeddings: '.format(np.sum(np.sum(embedding_matrix, axis=1) == 0)))


In [ ]:
X = list(zip(X_1, X_2, train_df['q1_n_words'].values, train_df['q2_n_words'].values, train_df['have_same_q_w'].values ))

In [ ]:
import pickle
pickle.dump( X, open( "X.p", "wb" ) )
pickle.dump( y, open( "y.p", "wb" ) )
pickle.dump( embedding_matrix, open( "embedding_matrix.p", "wb" ) )

In [ ]:
q1 = Input(shape=(sequence_length,))
q2 = Input(shape=(sequence_length,))
q1_n_words = Input(shape=(1,))
q2_length = Input(shape=(1,))
emb = Embedding(input_dim = embedding_matrix.shape[0], output_dim = embedding_matrix.shape[1], 
                            weights=[embedding_matrix],input_length = sequence_length, trainable=False)
lstm = Bidirectional(LSTM(256 ,dropout=0.5, return_sequences = True))

e1 = emb(q1)
l1 = lstm(e1)

e2 = emb(q2)
l2 = lstm(e2)

atten_1 = Attention(sequence_length)(l1)
atten_2 = Attention(sequence_length)(l2)
concat = Concatenate()([atten_1,atten_2,q1_length, q2_length])
output = Dense(300, activation="relu")(concat)
output = BatchNormalization()(concat)
output = Dropout(0.3)
output = Dense(150, activation="relu")(concat)

output = Dense(1, activation="sigmoid")(output)
